In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

path = "다운로드.jpg"

image = np.array(Image.open(path).convert("RGB").resize((256, 256)))
image_r, image_g, image_b = image[:,:,0], image[:,:,1], image[:,:,2]
image_s = np.stack([image_r, image_g, image_b], -1)
image_r, image_g, image_b = np.diag(image_r), np.diag(image_g), np.diag(image_b)

# image = plt.imshow(image)
# plt.show()

# image_s = plt.imshow(image_s)
# plt.show()

In [ ]:
import cv2
import math
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from torchvision import transforms

video_legal_dir = r"D:\Video-Dataset\2022-NSR-3m-재분류\agree"
video_illegal_dir = r"D:\Video-Dataset\2022-NSR-3m-재분류\non-agree"

transform = transforms.Compose([
    transforms.ToTensor()
])

def extract_diagnoal_matrix(dir_path : str):
    video_paths = glob.glob(dir_path + "\*")[:20]
    video_paths.sort()
    video_diagonals = []
    video_names = []

    for video_path in tqdm(video_paths, total=len(video_paths), desc="Extract Features"):
        vidcap = cv2.VideoCapture(video_path)
        vid_total_frames_num = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
        vid_frame_per_s = int(vidcap.get(cv2.CAP_PROP_FPS))

        video_name = video_path.split("\\")[-1]

        sections, retstep = np.linspace(1, vid_total_frames_num, 256, retstep=True)
        sections = list(map(math.floor, sections))
        frame_diagonals = []

        while(vidcap.isOpened()):
            ret, frame = vidcap.read()
            
            if not ret:
                break
            
            if int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) in sections:
                # frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
                frame = cv2.resize(frame, (256, 256))

                frame_r, frame_g, frame_b = frame[:,:,0], frame[:,:,1], frame[:,:,2]
                frame_r = [[int(np.mean(row))] for row in frame_r]
                frame_g = [[int(np.mean(row))] for row in frame_g]
                frame_b = [[int(np.mean(row))] for row in frame_b]
                # frame_r, frame_g, frame_b = np.diag(frame_r), np.diag(frame_g), np.diag(frame_b)
                frame_diagonal = np.stack([frame_r, frame_g, frame_b], -1)
                # frame_diagonal = np.expand_dims(frame_diagonal, 1)

                frame_diagonals.append(frame_diagonal)
        
        vidcap.release()
        video_diagonal = np.concatenate(frame_diagonals, axis=1)
        video_diagonals.append(video_diagonal)
        video_names.append(video_name)
    
    return video_names, video_diagonals

legal_diagonals = extract_diagnoal_matrix(video_legal_dir)
illegal_diagonals = extract_diagnoal_matrix(video_illegal_dir)

In [ ]:
import torch

diagonals_legal = []
diagonals_illegal = []
for diagonal_legal, diagonal_illegal in zip(legal_diagonals[1], illegal_diagonals[1]):
    diagonals_legal.append(diagonal_legal)
    diagonals_illegal.append(diagonal_illegal)

In [ ]:
for i in range(5):    
    plt.imshow(diagonals_illegal[i+15])
    plt.show()

In [6]:
from sklearn.model_selection import train_test_split
a = [(0,1),(0,1),(1,0),(1,0),(1,0),(1,0),(1,0),(1,0),(1,0),(1,0)]

x, y = train_test_split(a, test_size=0.2, shuffle=True, random_state=34)
print(x, y)

[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1)] [(0, 1), (1, 0)]


In [12]:
import random
a = [0,1,2,3,4,5]
random.shuffle(a)
a

[4, 2, 0, 5, 3, 1]

In [ ]:
plt.imshow(legal_diagonals[1][0])
plt.show()

# plt.imshow(legal_diagonals[2])
# plt.show()

In [ ]:
from PIL import Image

save_dir = r"D:\Video-Dataset\2022-NSR-3m-재분류\non-agree-image"

def image_save(images):
    for name, image in tqdm(zip(images[0], images[1]), total=len(images[0]), desc="Image save"):
        name = name.split(".")[0] + ".png"
        image = Image.fromarray(image)
        image.save(f"{save_dir}\{name}")

# image_save(legal_diagonals)
image_save(illegal_diagonals)